![](imgs/deepsense_header.png)

# Machine Learning and Big Data

A course by [deepsense.io](http://deepsense.io/).

# Part 2: Linear Regression 

A simple yet powerful tool for predicting numerical values.

* Fast
* Easy to start
* With an easy interpretation
* Can be used for variable selection
* Can be tweaked a lot

![](imgs/wikipedia_correlation.png)

In [ ]:
# dataframes
import pandas as pd

# plots
import seaborn as sns

# some other plots
import matplotlib.pyplot as plt

# so the plots appear inline
%matplotlib inline

# numerics (e.g. logarithm)
import numpy as np

# regression models from scikit-learn
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.ensemble import RandomForestRegressor

# model evaluation
from sklearn.cross_validation import train_test_split, KFold

In [ ]:
# loading data
df = pd.read_csv("data/Bike-Sharing-Dataset/day.csv", parse_dates=["dteday"], index_col="dteday")

# removing outliers (Linear Regression is sensitive to them!)
df = df.query("temp - atemp < 0.2").query("cnt > 100") 

## Linear Regression for 2 variables

The simplest case for Linear Regression - using a single variable to predict another.

In [ ]:
# sns.jointplot allows us to plot two variables as scatterplot with histograms
# option `kind='reg'` adds a regression line
sns.jointplot('temp', 'atemp', df, kind='reg')

In [ ]:
# some other pairs are less correlated
sns.jointplot('casual', 'atemp', df, kind='reg')

In [ ]:
# input
X = df[['temp']]

# output
Y = df['atemp']

In [ ]:
# X is a DataFrame with one column
# in general, we can use as many columns as we wish
X.head()

In [ ]:
# Y is a Series (just a column)
Y.head()

In [ ]:
# creating a Linear Regression object
reg = LinearRegression()

In [ ]:
# training classifier with data 
reg.fit(X, Y)

In [ ]:
# linear regression coefficient
reg.coef_

In [ ]:
# linear regression constant term
reg.intercept_

It means that we use the following formula:

$$\text{atemp} = 0.887 \cdot \text{temp} + 0.037$$

In [ ]:
# predict atemp for temp=0.5
reg.predict(0.5)

In [ ]:
# predict atemp for temps: 0.5, 0.2 and -0.3
reg.predict([[0.5], [0.2], [-0.3]])

## Testing

Always we need to test our predictions.

* plotting
* cross validation

In [ ]:
# predictions for the actual data
Y_pred = reg.predict(X)

In [ ]:
df_pred = pd.DataFrame({"actual": Y, "predicted": Y_pred})
df_pred.head()

In [ ]:
sns.jointplot('actual', 'predicted', df_pred)

In [ ]:
# R^2
# from 0 (predicting mean value) to 1 (perfect prediction)
# it CAN be negative for predictions being worse than guessing 
reg.score(X, Y)

In [ ]:
# cross-validation
# splitting, where 75% data goes for training and 25% for testing the prediction
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

In [ ]:
# we fit on the training set
reg = LinearRegression()
reg.fit(X_train, Y_train)  

# we can test on the same set
print("R^2 on the train set:")
print(reg.score(X_train, Y_train))

# but the actual predictive value is measured by the test set, never seen during fitting
print("\nR^2 on the test set:")
print(reg.score(X_test, Y_test))

### Exercise

* Modify the above (starting from the lines defining `X` and `Y`) so it predicts `casual` (the number of casual users) based on `atemp` (the feeling temperature).

## More variables

Almost always we want to use more parameters than one.

In [ ]:
# input
X = df[['temp', 'atemp', 'hum', 'windspeed', 'weathersit', 'weekday']].copy()

# output
Y = df['casual']

# splitting for cross-validation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

# regression
reg = LinearRegression()
reg.fit(X_train, Y_train)

print("R^2 on the train set:")
print(reg.score(X_train, Y_train))

print("\nR^2 on the test set:")
print(reg.score(X_test, Y_test))

In [ ]:
df_pred = pd.DataFrame({"actual_casual": Y, "predicted_casual": reg.predict(X)})
sns.jointplot('actual_casual', 'predicted_casual', df_pred)

In [ ]:
# plotting the importance of variables
coeffs = pd.Series(reg.coef_, index=X.columns)
coeffs_normalized = coeffs * X.std() / Y.std()
coeffs_normalized.plot(kind="barh")

### Exercise

* Removing which column in `X` changes the result the most?

## K-fold cross-validation

Note that a single random train test split can produce unstable scores. We will perform multiple splits by [K-fold cross-validation](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.KFold.html#sklearn.cross_validation.KFold) to account for this.

In [ ]:
reg = LinearRegression()
cv = KFold(n = X.shape[0], n_folds=10, shuffle=True)

In [ ]:
test_scores = []
for train_index, test_index in cv:
    X_train = X.iloc[train_index] 
    Y_train = Y.iloc[train_index]
    X_test = X.iloc[test_index] 
    Y_test = Y.iloc[test_index]
    # regression
    reg.fit(X_train, Y_train)
    
    print("---------------------")
    print("R^2 on the train set:")
    print(reg.score(X_train, Y_train))

    print("\nR^2 on the test set:")
    score = reg.score(X_test, Y_test)
    print(score)
    test_scores.append(score)

In [ ]:
np.mean(test_scores)

## Transforming variables

Sometimes transforming variables improves fit.

* logarithmic scale
* dummy variables
* normalization

In [ ]:
np.sqrt(9)

In [ ]:
np.log10(1000)

### Exercise

* What is the logarithm (base 10) of: `1000000`, `0.001`, `-3`?

In [ ]:
# some artificial data
v1 = np.random.rand(100)
v2 = np.exp(2 * v1 + 0.2 * np.random.randn(100))
v3 = np.exp(4 * v1 + 0.3 * np.random.randn(100))

In [ ]:
plt.plot(v1, v2, 'o')

In [ ]:
# np.log10 variable[s]
plt.plot(v3, v1, 'o')

In [ ]:
# np.log10 variable[s]
plt.plot(v2, v3, 'o')

### Exercise

* Apply logarithm for some variables (i.e write `np.log10(vx)` instead of `vx`) when it makes the plot more linear.

In [ ]:
# we copy a dataframe, so we can safely modify it
df_t = df.copy()

In [ ]:
df_t.plot(kind='scatter', x='temp', y='casual')

In [ ]:
df_t.plot(kind='scatter', x='temp', y='casual', logy=True)

In [ ]:
# this plot may be convenient for testing transformations
plt.plot(df_t["atemp"], np.sqrt(df_t["casual"]), 'o')

In [ ]:
# logarithmic scalling of a variable
df_t["casual"].apply(np.log10).hist(bins=50)

In [ ]:
# changing a variables for good
df_t["casual"] = np.log10(df_t["casual"])

In [ ]:
# so it is converted
df_t["casual"].hist(bins=50)

### Dummy variables

In [ ]:
# changing categorical variables into binary
pd.get_dummies(df_t[["temp", "weekday"]], columns=['weekday']).head(10)

### Normalization

In [ ]:
# normalzation
df_t["cnt_norm"] = (df["cnt"] - df["cnt"].mean()) / df["cnt"].std()

In [ ]:
# it has mean 0 (up to numerical precission)
df_t["cnt_norm"].mean()

In [ ]:
# and standard deviation 1 (again, up to the  numerical precission)
df_t["cnt_norm"].std()

### Transformations - and regression

In [ ]:
# let's see how it works in practice

# input
X = df[['temp', 'atemp', 'hum', 'windspeed', 'weathersit', 'weekday']].copy()
X = pd.get_dummies(X, columns=['weekday'])
X = X.drop('weekday_1', axis=1)

# output
Y = df['casual'].apply(np.log10)

# splitting for cross-validation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

# regression
reg = LinearRegression()
reg.fit(X_train, Y_train)

print("R^2 on the train set:")
print(reg.score(X_train, Y_train))

print("\nR^2 on the test set:")
print(reg.score(X_test, Y_test))

In [ ]:
# plotting the importance of variables
coeffs = pd.Series(reg.coef_, index=X.columns)
coeffs_normalized = coeffs * X.std() / Y.std()
coeffs_normalized.plot(kind="barh")

In [ ]:
df_pred = pd.DataFrame({"actual_casual": Y, "predicted_casual": reg.predict(X)})
sns.jointplot('actual_casual', 'predicted_casual', df_pred)

### Exercise

* Try turning dummy variables on and off.
* Try `np.sqrt` instead of `np.log10`.

It is better? Worse?

### A note on the logarithm

For a positive variable, we can run regression on its logarithm rather original value. Then the fitting reads: 

$$\ln(y) = a_1 x_1 + a_2 x_2 + a_3 x_3 + b$$

The result can be transformed into

$$y = B A_1^{x_1} A_2^{x_2} A_3^{x_3}$$

so:

  * $y$ is always positive,
  * each factor $x_i$ *multiplies* rather than *adds to* the result

## Interactions

Sometimes we need to take into account interactions between two (or more) variables. Interactions are applicable when we want to model extra dependency between variables. They are modelled by introducing pairwise feature multiplications in a model:

$$y = b + a_1 x_1 + a_2 x_2 + \boldsymbol{a_3 x_1 x_2}$$

This can be rewritten as 

$$y = b + a_1 x_1 + \boldsymbol{(a_2 + a_3 x_1)x_2}.$$

We may interpret it as **a change of the influence** of variable $x_2$ on $y$ **depending on** the level of variable $x_1$. 

In [ ]:
win = [('win', 'yes'), ('win', 'no')]
son = [('scored', 'yes'), ('scored', 'no')]

In [ ]:
interactions = pd.DataFrame([[20, 5], [3, 0]], index=pd.MultiIndex.from_tuples(win), columns=pd.MultiIndex.from_tuples(son))
interactions

And back to the real data:

In [ ]:
df_int = df.copy()
for col in df:
    df_int[col + "_x_" "workingday"] = df[col] * df["workingday"]

In [ ]:
df_int.head()

In [ ]:
# input
X = df[['temp', 'atemp', 'hum', 'windspeed', 'weathersit', 'weekday']].copy()
X = pd.get_dummies(X, columns=['weekday'])
X = X.drop('weekday_2', axis=1)

cols = X.columns 

for col1 in cols:
    for col2 in cols:
        if col1 <= col2 and not (col1.startswith("weekday") and col2.startswith("weekday")):
            X[col1 + '_x_' + col2] = X[col1] * X[col2]
            
# output
Y = df['casual'].apply(np.log10)

# splitting for cross-validation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=15)

# regression
reg = LinearRegression()
reg.fit(X_train, Y_train)

print("R^2 on the train set:")
print(reg.score(X_train, Y_train))

print("\nR^2 on the test set:")
print(reg.score(X_test, Y_test))

## Linear Regression with  LASSO Regularization

* still Linear Regression
* tries to avoid spurious correlations (overfitting)

Typical Linear Regression minimizes square deviation from  (so-called least-square fitting)

$$\sum_i (y_i - \bar{y}_i)^2$$

LASSO Linear Regression minimizes the above plus the absolute value of all regression coefficients

$$\sum_i (y_i - \bar{y}_i)^2 + \lambda \sum_j |a_j|$$

In [ ]:
# input
X = df[['temp', 'atemp', 'hum', 'windspeed', 'weathersit', 'weekday']].copy()
X = pd.get_dummies(X, columns=['weekday'])
# X = X.drop('weekday_2', axis=1)  # we don't need that in LASSO

cols = X.columns 

for col1 in cols:
    for col2 in cols:
        if col1 <= col2 and not (col1.startswith("weekday") and col2.startswith("weekday")):
            X[col1 + '_x_' + col2] = X[col1] * X[col2]
             
# output
Y = df['casual'].apply(np.log10)

# splitting for cross-validation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=15)

# LASSO Linear Regression instead of an ordinary one
lasso = LassoCV(normalize=True, max_iter=10000)
lasso.fit(X_train, Y_train)

print("R^2 on the train set:")
print(lasso.score(X_train, Y_train))

print("\nR^2 on the test set:")
print(lasso.score(X_test, Y_test))

In [ ]:
# LASSO turns many coefficients in zeros
lasso.coef_

In [ ]:
# plotting the importance of variables
coeffs = pd.Series(lasso.coef_, index=X.columns)
coeffs.plot(kind="bar", figsize=(12, 6))

### Exercises

* Use in the above:
    * remove interactions,
    * use `season` and `weathersit` with dummy variables,
    * ★ add interaction between only between `workingday` and other variables.
* ★ Predict `casual` users based on `registered`  (but not `cnt`) and `workingday`.

## Random Forest Regression

Let's try a different model - [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor). It works for both regression and classification.

In [ ]:
# creating a random forest regressor
rfr = RandomForestRegressor(n_estimators=50, oob_score=True)

In [ ]:
X = df.drop(["cnt", "mnth", "registered", "casual", "instant"], axis=1)
Y = df["casual"]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

In [ ]:
rfr.fit(X_train, Y_train)

In [ ]:
# don't do that!
# for Random Forest score on the training set is (almost) always very high
rfr.score(X_train, Y_train)

In [ ]:
# on test set it is lower
rfr.score(X_test, Y_test)

In [ ]:
# alternatively, instead of cross validating
# we can estimate score by measuring so-called out-of-box score
rfr.oob_score_

In [ ]:
# feature (variable) importance
rfr.feature_importances_

In [ ]:
# but we want to plot it
pd.Series(rfr.feature_importances_, index=X.columns).plot(kind="barh")

### Exercises

* What is the feature importance if you remove:
    * `temp`?
    * `temp` and `atemp`?